# 알고리즘 및 실습 실습과제 8

## [문제 1] 교재 5장 연습문제 문제 22번, 23번 풀기.
- closest_dist_pair() 알고리즘 개선
- 복잡도 분석
### [문제 1, 229P-22] closest_pair_dist() 알고리즘을 $O(n log_2 n)$으로 개선하라. 이를 위해, strip_closest()에서 정렬 문장을 제거해야 하고, closest_pair_dist()에서 병합 정렬의 병합 기법(알고리즘 5.2)을 사용해야 할 것이다.

In [3]:
import math
import time
import random


# p1, p2 두 점 사이의 거리를 구하기
def distance(p1, p2):
    return ((p1[0] - p2[0]) ** 2 + (p1[1] - p2[1]) ** 2) ** 0.5


# 억지 기법으로 최근접 쌍 거리 구하기
def closest_pair_bruteforce(P):
    n = len(P)
    min_dist = float("inf")
    for i in range(n - 1):
        for j in range(i + 1, n):
            dist = distance(P[i], P[j])
            # 모든 점에 대해서 순회하며 거리를 구하기
            if dist < min_dist:
                min_dist = dist
                # 현재 최소 거리 값보다 현재 거리가 작으면 최소 값 갱신
    return min_dist


# 두 리스트를 한 리스트로 병합하여 반환
def merge(left, right):
    merged = []
    # 병합 리스트
    while left and right:
        # left와 right의 원소가 있으면
        if left[0][1] < right[0][1]:
            # Y 좌표 기준으로 병합
            merged.append(left.pop(0))
            # left의 최좌측 원소를 빼서 merged에 넣기
        else:
            merged.append(right.pop(0))
            # right의 최좌측 원소를 빼서 merged에 넣기

    if left:
        # left의 원소가 남은 경우
        merged.extend(left)
        # 남은 원소 병합 리스트에 추가
    else:
        # right의 원소가 남은 경우
        merged.extend(right)
        # 남은 원소 병합 리스트에 추가
    return merged


# y 기준으로 정렬된 리스트의 띠 내부에서 최소인 거리 구하기
def strip_closest_without_sort(P, d):
    n = len(P)
    # 리스트 내의 점의 수
    d_min = d

    for i in range(n):  # y가 최소인 점부터 순서대로
        j = i + 1
        # P[i].y와 P[j].y의 차이가 d_min 이내일 때까지만 처리
        while j < n and (P[j][1] - P[i][1]) < d_min:
            dij = distance(P[i], P[j])
            if dij < d_min:
                d_min = dij
            j += 1
    return d_min


def get_runway(ysorted_points, medianx, delta):
    return [pt for pt in ysorted_points if medianx - delta <= pt[0] <= medianx + delta]


def _closest_pair_with_merge(P, n):
    if n <= 3:
        return closest_pair_bruteforce(P), sorted(P, key=lambda point: point[1])
        # 억지 기법으로 최근접쌍 구하기, P를 정렬하여 반환하기

    mid = n // 2
    # 중앙점 찾기
    left_list = P[:mid]
    right_list = P[mid:]
    # 좌, 우 리스트로 분할

    mid_x = P[mid][0]
    # 중앙점의 X 좌표

    dl, left_list_y_sorted = _closest_pair_with_merge(left_list, mid)
    # 분할된 좌 리스트에서 최근접 쌍 거리를 구하고, y를 기준으로 정렬
    dr, right_list_y_sorted = _closest_pair_with_merge(right_list, n - mid)
    # 분할된 우 리스트에서 최근접 쌍 거리를 구하고, y를 기준으로 정렬

    d = min(dl, dr)
    # 좌, 우 리스트에서 가장 작은 점 구하기

    merged_lists = merge(left_list_y_sorted, right_list_y_sorted)
    # 좌, 우 리스트를 병합 처리하기

    Pm = []
    for i in range(n):  # Pm도 x에 대해 정렬되어 있음
        if abs(merged_lists[i][0] - mid_x) < d:
            Pm.append(merged_lists[i])
        # x 차이가 d보다 작은 리스트만 필터링
    return strip_closest_without_sort(Pm, d), merged_lists


# 병합을 이용한 최근접 쌍 구하기 래퍼 함수
def closest_pair_with_merge(P):
    P = sorted(P, key=lambda point: point[0])
    dist, P = _closest_pair_with_merge(P, len(P))
    return dist


TRY = 100

merge_total_time = 0
bruteforce_total_time = 0

# 1000개 좌표의 리스트를 100번 테스트한 시간 측정
for _ in range(TRY):
    test_set = list(set([tuple(random.randint(0, 10000) for _ in range(2)) for _ in range(1000)]))
    t1 = time.time()
    min_bruteforce = closest_pair_bruteforce(test_set)
    t2 = time.time()
    min_merge = closest_pair_with_merge(test_set)
    t3 = time.time()
    bruteforce_total_time += t2 - t1
    merge_total_time += t3 - t2
    if min_bruteforce != min_merge:
        print("WARNING: The results are different.")
        print("min_bruteforce:", min_bruteforce)
        print("min_merge:", min_merge)
        print()

print("bruteforce total time:", bruteforce_total_time)
print("merge total time:", merge_total_time)

bruteforce total time: 45.189157247543335
merge total time: 0.842109203338623


### [문제 1, 229P-23] 위에서 수정한 알고리즘의 시간 복잡도를 계산하라.
`closest_pair_with_merge`의 전처리 부분의 시간 복잡도 (x 좌표 기준 정렬, TimSort 이용) = $O(n log _2n)$

#### `_closest_pair_with_merge`의 시간복잡도 계산
복잡도 함수를 구하기 위해서 함수 내부의 계산 요소들을 분석하면

P의 길이가 3 이하인 리스트에 대해서 `closest_pair_bruteforce` 호출 = $O(1)$

`mid`, `left_list`, `right_list`, `mid_x`등의 계산 변수 값 초기화 = $O(1)$

...

`merge` 함수로 좌, 우 리스트 병합 = $O(n)$

Pm 계산과 `strip_closest_without_sort` 함수 호출 = $O(n)$

중간의 분할 과정을 고려하여  `_closest_pair_with_merge` 함수의 복잡도 함수를 다음과 같이 표기할 수 있다.

$
T(n)=
    \begin{cases}
        const & n=1 \\
        T(n)=2T(\frac{n}{2}) + n + const & otherwise \\
    \end{cases}
$
$
T(n)=
    \begin{cases}
        O(n^d) & if\ a<b^d \\
        O(n^d log n) & if\ a=b^d \\
        O(n^{log _b a}) & if\ a>b^d \\
    \end{cases}
$

마스터 정리에 의해서 (a=2, b=2, d=1이므로) $T(n)$의 시간 복잡도는 다음과 같이 나타낼 수 있다.

$O(T(n))=O(n log n)$